In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('reviews_dataset2.csv')
print(df.head())

texts = df['text'].values
labels = df['label'].values

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts).toarray()

label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(labels)

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

                                                text  label
0  one best crichton novel sphere michael crichto...      1
1  medicine future z accomplished heart surgeon f...      1
2  beautiful gorgeous network comic book contains...      1
3  lover robicheaux book lover robicheaux demon s...      1
4  excellent broad survey development civilizatio...      1


In [ ]:
# simple neuron network

import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

# create
class SimpleNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return self.softmax(x)

model = SimpleNN(input_dim=x_train.shape[1], hidden_dim=1000, output_dim=len(set(Y)))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# train
for epoch in range(100):
    model.train()
    inputs = torch.tensor(x_train, dtype=torch.float32)
    targets = torch.tensor(y_train, dtype=torch.long)

    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()

    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# estimating
model.eval()
with torch.no_grad():
    inputs = torch.tensor(_test, dtype=torch.float32)
    targets = torch.tensor(y_test, dtype=torch.long)
    outputs = model(inputs)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == targets).float().mean()
    print(f'Test Accuracy: {accuracy.item()}')